In [1]:
import pandas as pd

In [2]:
event_name = '[TEDxNations]'

In [14]:
event_data = pd.read_table('data/%s_data_clean_annotated.txt' % event_name, sep='\t', header=0, encoding='utf-8', index_col=0)

In [15]:
event_data.shape

(986, 1)

In [17]:
event_data['Class'].describe()

count    263.000000
mean       2.342205
std        0.491149
min        1.000000
25%        2.000000
50%        2.000000
75%        3.000000
max        3.000000
Name: Class, dtype: float64

In [18]:
event_data['Class'].value_counts()

2    169
3     92
1      2
dtype: int64